In [1]:
import os
import numpy as np
from dfply import *
import pandas as pd
from datetime import datetime

In [2]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format

In [6]:
# 데이터 불러오기
자동차_상품군정보 = pd.read_excel('./data/자동차_현행추정부채/자동차_상품군정보.xlsx', sheet_name='상품군분류', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str})
자동차_분류 = pd.read_excel('./data/자동차_현행추정부채/자동차_상품군정보.xlsx', sheet_name='KICS분류')
금감원_금리시나리오 = pd.read_excel('./data/자동차_현행추정부채/금감원_금리시나리오.xlsx') # 상승/하락/평행/경사 구분코드 넣기
자동차_원수_개별추산액 = pd.read_excel('./data/자동차_현행추정부채/자동차_원수_개별추산액.xlsx', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
자동차_원수_미경과보험료 = pd.read_excel('./data/자동차_현행추정부채/자동차_원수_미경과보험료.xlsx', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
자동차_납입예정보험료 = pd.read_excel('./data/자동차_현행추정부채/자동차_납입예정보험료.xlsx', dtype={'CLG_YM': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'PYM_EPCT_YM': str})
자동차_출재_비례_미경과보험료 = pd.read_excel('./data/자동차_현행추정부채/자동차_출재_비례_미경과보험료.xlsx', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
자동차_출재_비례_개별추산액 = pd.read_excel('./data/자동차_현행추정부채/자동차_출재_비례_개별추산액.xlsx', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
자동차_출재_비례_직전1년경과보험료 = pd.read_excel('./data/자동차_현행추정부채/자동차_출재_비례_직전1년경과보험료.xlsx', dtype={'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDGR_CD': str, 'PDC_CD': str, 'BSC_CVR_CD': str})
자동차_출재_비비례_직전1년경과보험료 = pd.read_excel('./data/자동차_현행추정부채/자동차_출재_비비례_직전1년경과보험료.xlsx', dtype={'CLG_YM': str, 'BSC_CVR_CD': str})
자동차_진전추이 = pd.read_excel('./data/자동차_현행추정부채/자동차_진전추이.xlsx', dtype={'BSE_YM': str, 'AY_YM': str})

#### 1. 자동차/원수/계약 - 미경과보험료

In [35]:
# 자동차 PDGR_NM 가공
자동차_원수_미경과보험료_가공 = 자동차_원수_미경과보험료 >> left_join(자동차_상품군정보, on=[X.PDGR_CD, X.BSC_CVR_CD]) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM))

In [36]:
 자동차_원수_미경과보험료_집계 = 자동차_원수_미경과보험료_가공 >> group_by(X.PDGR_NM) \
    >> summarise(OGL_URND_PRM=np.sum(X.OGL_URND_PRM)) \
    >> mutate(OGL_URND_PRM=X.OGL_URND_PRM/1e3)
자동차_원수_미경과보험료_집계 = 자동차_원수_미경과보험료_집계.reset_index(drop=True)
자동차_원수_미경과보험료_집계

,PDGR_NM,OGL_URND_PRM
0,개인용자동차(물담보),"711,247,077"
1,개인용자동차(인담보),"425,354,487"
2,업무용자동차(물담보),"210,669,119"
3,업무용자동차(인담보),"142,884,079"
4,영업용자동차(물담보),"12,181,875"
5,영업용자동차(인담보),"48,234,053"
6,자동차기타,"78,921,777"


#### 2. 자동차/출재(비례)/계약 - 미경과보험료

In [37]:
자동차_출재_비례_미경과보험료_가공 = 자동차_출재_비례_미경과보험료 >> left_join(자동차_상품군정보, on=[X.PDGR_CD, X.BSC_CVR_CD]) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM))

In [38]:
 자동차_출재_비례_미경과보험료_집계 = 자동차_출재_비례_미경과보험료_가공 >> group_by(X.PDGR_NM) \
    >> summarise(RN_URND_PRM=np.sum(X.RN_URND_PRM)) \
    >> mutate(RN_URND_PRM=X.RN_URND_PRM/1e3)
자동차_출재_비례_미경과보험료_집계 = 자동차_출재_비례_미경과보험료_집계.reset_index(drop=True)
자동차_출재_비례_미경과보험료_집계

,PDGR_NM,RN_URND_PRM
0,개인용자동차(물담보),"11,164"
1,개인용자동차(인담보),"5,881"
2,업무용자동차(물담보),"5,884,417"
3,업무용자동차(인담보),"4,139,076"
4,자동차기타,0


#### 3. 할인율

In [46]:
금감원_금리시나리오_가공 = 금감원_금리시나리오 \
    >> mutate(MATURITY=X.BUCKET_END_DATE.dt.year*12 + X.BUCKET_END_DATE.dt.month - X.AS_OF_DATE.dt.year*12 - X.AS_OF_DATE.dt.month) \
    >> mutate(DISCOUNT=1/(1+X.VALUE)**(X.MATURITY/12)) \
    >> select(X.MATURITY, X.DISCOUNT)

#### 4. 자동차/원수/계약 - 미래보험료

In [62]:
# 자동차 계약 납입예정보험료 MATURITY 가공
자동차_납입예정보험료_가공 = 자동차_납입예정보험료 >> left_join(자동차_상품군정보, on=['PDGR_CD', 'BSC_CVR_CD']) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM)) \
    >> mutate(PYM_EPCT_YM=pd.to_datetime(자동차_납입예정보험료['PYM_EPCT_YM'], format='%Y%m')) \
    >> mutate(BSE_YM=pd.to_datetime(자동차_납입예정보험료['CLG_YM'], format='%Y%m')) \
    >> mutate(MATURITY=X.PYM_EPCT_YM.dt.year*12 + X.PYM_EPCT_YM.dt.month - X.BSE_YM.dt.year*12 - X.BSE_YM.dt.month) \
    >> drop(X.BSE_YM, X.PYM_EPCT_YM) \
    >> mutate(MATURITY=if_else(X.MATURITY < 0, 0, X.MATURITY))

# 자동차 할인율 반영
자동차_납입예정보험료_가공 = 자동차_납입예정보험료_가공 >> left_join(금감원_금리시나리오_가공, on='MATURITY') \
    >> mutate(WNCR_CNV_EPCT_PRM_DISC=X.EPCT_PRM*X.DISCOUNT) \
    >> mutate(WNCR_CNV_RN_PRM_DISC=X.RN_PRM*X.DISCOUNT)

In [65]:
# 자동차 계약 납입예정보험료 집계
자동차_납입예정보험료_집계 = 자동차_납입예정보험료_가공 >> group_by(X.PDGR_NM) \
    >> summarise(WNCR_CNV_EPCT_PRM_DISC=np.sum(X.WNCR_CNV_EPCT_PRM_DISC), WNCR_CNV_RN_PRM_DISC=np.sum(X.WNCR_CNV_RN_PRM_DISC)) \
    >> mutate(WNCR_CNV_EPCT_PRM_DISC=X.WNCR_CNV_EPCT_PRM_DISC/1e3, WNCR_CNV_RN_PRM_DISC=X.WNCR_CNV_RN_PRM_DISC/1e3)
자동차_납입예정보험료_집계 = 자동차_납입예정보험료_집계.reset_index(drop=True)
자동차_납입예정보험료_집계

,PDGR_NM,WNCR_CNV_EPCT_PRM_DISC,WNCR_CNV_RN_PRM_DISC
0,개인용자동차(물담보),"1,988,314",0
1,개인용자동차(인담보),"1,415,501",0
2,업무용자동차(물담보),"472,687","13,238"
3,업무용자동차(인담보),"501,941","16,768"
4,영업용자동차(물담보),"3,370,412",0
5,영업용자동차(인담보),"7,594,403",0
6,자동차기타,"10,409,947",0


#### 5. 자동차/원수/보상 - 개별추산액

In [108]:
# 자동차 PDGR_NM 가공
자동차_원수_개별추산액_가공 = 자동차_원수_개별추산액 >> left_join(자동차_상품군정보, on=[X.PDGR_CD, X.BSC_CVR_CD]) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM))

In [110]:
자동차_원수_개별추산액_집계 = 자동차_원수_개별추산액_가공 >> group_by(X.PDGR_NM) \
    >> summarise(OST_AMT=np.sum(X.OST_AMT)) \
    >> mutate(OST_AMT=X.OST_AMT/1e3)
자동차_원수_개별추산액_집계 = 자동차_원수_개별추산액_집계.reset_index(drop=True)
자동차_원수_개별추산액_집계

,PDGR_NM,OST_AMT
0,개인용자동차(물담보),"71,221,844"
1,개인용자동차(인담보),"261,346,647"
2,업무용자동차(물담보),"22,135,578"
3,업무용자동차(인담보),"108,916,199"
4,영업용자동차(물담보),"2,290,806"
5,영업용자동차(인담보),"46,671,633"
6,자동차기타,"18,046,376"


#### 6. 자동차/출재/보상 - 개별추산액

In [115]:
# 자동차 출재 보상 PDGR_NM 가공
자동차_출재_비례_개별추산액_가공 = 자동차_출재_비례_개별추산액 >> left_join(자동차_상품군정보, on=[X.PDGR_CD, X.BSC_CVR_CD]) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM))

In [119]:
# 자동차 출재 보상 개별추산액 집계
자동차_출재_비례_개별추산액_집계 = 자동차_출재_비례_개별추산액_가공 >> group_by(X.PDGR_NM) \
    >> summarise(RN_OST_AMT=np.sum(X.RN_OST_AMT)) \
    >> mutate(RN_OST_AMT=X.RN_OST_AMT/1e3)
자동차_출재_비례_개별추산액_집계 = 자동차_출재_비례_개별추산액_집계.reset_index(drop=True)
자동차_출재_비례_개별추산액_집계

,PDGR_NM,RN_OST_AMT
0,개인용자동차(물담보),"125,785"
1,개인용자동차(인담보),"991,198"
2,업무용자동차(물담보),"596,688"
3,업무용자동차(인담보),"1,122,283"
4,영업용자동차(인담보),"50,145"
5,자동차기타,"533,481"


#### 7. 할인요소

In [100]:
자동차_진전추이_보종

,BSE_YM,AT_PDGR_CD,AY_YM,BASE_1,BASE_2,BASE_3,BASE_4,BASE_5,BASE_6,BASE_7,PDGR_NM
42,201712,B007,201112,12044654708,13898492845,14249591097,14459341542,14666412670,14654940327,14550714589,자동차기타
43,201712,B007,201212,13155574508,15467206563,15628546874,15723207323,15803726719,15872915370,15760027410,자동차기타
44,201712,B007,201312,15115421859,17971276153,18433442470,18556507178,18572357488,18607537107,18475200554,자동차기타
45,201712,B007,201412,16318130871,20069017102,20778806573,21041013743,21172011413,21212115277,21061254999,자동차기타
46,201712,B007,201512,23931081412,28349997447,28894702509,29183138851,29364827967,29420450602,29211212755,자동차기타
47,201712,B007,201612,26199605469,31103227784,31827278433,32144988705,32345117779,32406385657,32175911879,자동차기타
48,201712,B007,201712,26632094151,31644672352,32381327265,32704568233,32908181151,32970515580,32736029718,자동차기타


In [101]:
자동차_진전추이_가공 = 자동차_진전추이.copy()

할인요소 = []
for AT_PDGR_CD in 자동차_진전추이_가공['AT_PDGR_CD'].unique():
    자동차_진전추이_보종 = 자동차_진전추이_가공.query('AT_PDGR_CD == @AT_PDGR_CD').sort_values(by='AY_YM')
    할인요소_보종 = []
    m = len(자동차_진전추이_보종) - 1
    for i in range(m):
        n = m - i
        지급보험금 = 0
        for j in range(n):
            지급보험금 += 자동차_진전추이_보종.iloc[m-j, 3+j+1+i] - 자동차_진전추이_보종.iloc[m-j, 3+j+i]
        평균지급기간 = i + 0.5
        지급보험금 = max(지급보험금, 0)
        할인요소_보종.append([평균지급기간, 지급보험금])
    할인요소_보종 = pd.DataFrame(할인요소_보종, columns=['준비금_평균지급기간', '지급보험금'])
    할인요소_보종['지급보험금_비중'] = 할인요소_보종['지급보험금']/할인요소_보종['지급보험금'].sum()
#     할인요소_보종['누적비율'] = (할인요소_보종['지급보험금'] / 할인요소_보종['지급보험금'].sum()).cumsum()
    할인요소_보종['누적비율'] = 할인요소_보종['지급보험금_비중'].cumsum()
    할인요소_보종['MATURITY'] = (할인요소_보종['준비금_평균지급기간']*12).astype(int)
    할인요소_보종 = 할인요소_보종.merge(금감원_금리시나리오_가공, on='MATURITY', how='left')
    할인요소_보종 = 할인요소_보종.drop(['MATURITY'], axis=1)
    할인요소_보종.insert(0, 'AT_PDGR_CD', AT_PDGR_CD)
    할인요소_보종['보험료_평균지급기간'] = 할인요소_보종['준비금_평균지급기간'] - 0.2071
    할인요소_보종['조정비율'] = 0
    for i in range(m):
        if i==m-1:
            할인요소_보종['조정비율'].iloc[i] = 1
        else:
            할인요소_보종['조정비율'].iloc[i] = 할인요소_보종['누적비율'].iloc[i]+0.2071*(할인요소_보종['누적비율'].iloc[i+1]-할인요소_보종['누적비율'].iloc[i])
    할인요소_보종['1-조정비율'] = 0
    for i in range(m):
        if i==0:
            할인요소_보종['1-조정비율'].iloc[i] = 할인요소_보종['조정비율'].iloc[i]
        else:
            할인요소_보종['1-조정비율'].iloc[i] = 할인요소_보종['조정비율'].iloc[i]-할인요소_보종['조정비율'].iloc[i-1]
    할인요소_보종['MATURITY'] = np.round(할인요소_보종['보험료_평균지급기간']*12).astype(int)
    할인요소_보종 = 할인요소_보종.merge(금감원_금리시나리오_가공, on='MATURITY', how='left')
    할인요소_보종 = 할인요소_보종.drop(['MATURITY'], axis=1)
    할인요소.append(할인요소_보종)
할인요소 = pd.concat(할인요소).reset_index(drop=True)
할인요소 = 할인요소.merge(자동차_분류, on='AT_PDGR_CD', how='left').drop('AT_PDGR_CD', axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Prog

In [103]:
# 할인요소 집계
보험료부채_할인요소 = 할인요소.groupby('PDGR_NM').apply(lambda df: (df['지급보험금_비중']*df['DISCOUNT_x']).sum())
준비금부채_할인요소 = 할인요소.groupby('PDGR_NM').apply(lambda df: (df['1-조정비율']*df['DISCOUNT_y']).sum())
자동차_할인요소_집계 = pd.concat([보험료부채_할인요소, 준비금부채_할인요소], axis=1)
자동차_할인요소_집계.columns = ['할인요소_보험료', '할인요소_준비금']
자동차_할인요소_집계 = 자동차_할인요소_집계.reset_index()
자동차_할인요소_집계

,PDGR_NM,할인요소_보험료,할인요소_준비금
0,개인용자동차(물담보),1,1
1,개인용자동차(인담보),1,1
2,업무용자동차(물담보),1,1
3,업무용자동차(인담보),1,1
4,영업용자동차(물담보),1,1
5,영업용자동차(인담보),1,1
6,자동차기타,1,1


#### 8. 자동차/출재(비례)/계약 - 직전 1년 경과보험료

In [129]:
# 일반 출재 계약 PDGR_NM 가공
자동차_출재_비례_직전1년경과보험료_가공 = 자동차_출재_비례_직전1년경과보험료 >> left_join(자동차_상품군정보, by=['PDGR_CD', 'BSC_CVR_CD']) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM))

In [131]:
# 일반 출재 계약 직전1년경과보험료 집계
자동차_출재_비례_직전1년경과보험료_집계 = 자동차_출재_비례_직전1년경과보험료_가공 >> group_by(X.PDGR_NM) \
    >> summarise(RN_ELP_PRM=np.sum(X.RN_ELP_PRM).astype(float)) \
    >> mutate(RN_ELP_PRM=X.RN_ELP_PRM/1e3)
자동차_출재_비례_직전1년경과보험료_집계

,PDGR_NM,RN_ELP_PRM
0,개인용자동차(물담보),"4,373,067"
1,개인용자동차(인담보),"2,170,734"
2,업무용자동차(물담보),"4,813,128"
3,업무용자동차(인담보),"3,344,852"
4,영업용자동차(인담보),-96
5,자동차기타,"1,095,675"


#### 9 . 자동차/출재(비비례)/계약 - 직전 1년 경과보험료

In [141]:
자동차_출재_비비례_직전1년경과보험료_집계 = 자동차_출재_비비례_직전1년경과보험료 >> summarise(RN_ELP_PRM=np.sum(X.RN_ELP_PRM).astype(float)) \
    >> mutate(RN_ELP_PRM=X.RN_ELP_PRM/1e3)
자동차_출재_비비례_직전1년경과보험료_집계

,RN_ELP_PRM
0,"3,328,551"


In [142]:
if not any([s == 'result' for s in os.listdir('.')]):
    os.mkdir('result')
    
now = datetime.now().strftime('%Y%m%d%H%M%S')
ver = 'v1.1.0'
with pd.ExcelWriter('./result/자동차_현행추정부채_{}_{}.xlsx'.format(ver, now)) as writer:
    자동차_원수_미경과보험료_집계.to_excel(writer, '자동차_원수_미경과보험료', index=False)
    자동차_출재_비례_미경과보험료_집계.to_excel(writer, '자동차_출재_비례_미경과보험료', index=False)
    자동차_원수_개별추산액_집계.to_excel(writer, '자동차_원수_개별추산액', index=False)
    자동차_출재_비례_개별추산액_집계.to_excel(writer, '자동차_출재_비례_개별추산액', index=False)
    자동차_납입예정보험료_집계.to_excel(writer, '자동차_납입예정보험료', index=False)
    자동차_할인요소_집계.to_excel(writer, '자동차_할인요소', index=False)
    자동차_출재_비례_직전1년경과보험료_집계.to_excel(writer, '자동차_출재_비례_직전1년경과보험료', index=False)
    자동차_출재_비비례_직전1년경과보험료_집계.to_excel(writer, '자동차_출재_비비례_직전1년경과보험료', index=False)